In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

# Lab 10: Re-expression and Evaluating Models

Welcome to Lab 10! In this lab we will use re-expression to straighten a scatter plot to fit a simple linear model. Also, we will investigate methods to evaluate the "goodness of fit" for linear models.

To receive credit for a lab, answer all questions correctly and submit before the deadline.

**Due Date:** Tuesday, May 10, 2022 at 11:59 p.m.

**Collaboration Policy:** Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others **please include their names below** (it's a good way to learn your classmates' names).

**Collaborators:** 

List collaborators here.

**Note:** In this notebook a custom figure size has been configured. Click [here](https://matplotlib.org/users/customizing.html) to read the documentation about customizing aspects of matplotlib.

Run the cell below.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['figure.dpi'] = 100
from sklearn.linear_model import LinearRegression

# 1. Simple Linear Regression

By now you've most certainly noticed that we have been using the terms **predictors**, **features**, **independent variables**, and **explanatory variables** interchangeably. Usually "features" are used in the context of machine learning when you are trying to make predictions. "Independent variables" and "explanatory variables" are mainly found in statistics, econometrics, and other related fields which focus on understanding the relationship between a set of variables.  

For example, consider the plot below which shows fuel efficiency vs. engine power for several models of automobile.

**Note:** First, we load the full data set, drop any rows that have missing data and the data set by `horsepower` (in ascending order).

In [ ]:
mpg = sns.load_dataset('mpg').dropna()
mpg = mpg.sort_values('horsepower', ascending = True)
mpg.head()

# 2. Correlation Coefficient 

In Foundations of Data Science we learned about the correlation coefficient in section 15.1 of the textbook, *Computational and Inferential Thinking*. To refresh your memory about the concept click [here](https://inferentialthinking.com/chapters/15/1/Correlation.html). 

Next, let's take a look at the correlation between `mpg` and the other numeric variables.

In [ ]:
mpg.loc[:,['mpg', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']].corr()

<!-- BEGIN QUESTION -->

**Question 1.** Are there any independent variables that are highly correlated? If so, which ones? Does it seem reasonable that they would be? Write 2-3 sentences to respond to the aforementioned questions.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

Recall when we looked at [Anscombe's](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) quartet. Anscombe's quartet comprises four data sets that have nearly identical simple descriptive statistics, yet have very different distributions and appear very different when graphed. After that experience we decided that its a good idea to make a scatter plot to visualize the data and not rely solely on the correlation coefficient.

Run the cell below to see a scatter plot of the `horsepower` vs. `mpg`.

In [ ]:
sns.scatterplot(x = 'horsepower', y = 'mpg', data = mpg)
plt.xlabel("Horsepower")
plt.ylabel("mpg");

<!-- BEGIN QUESTION -->

A scatter plot of the data reveals that there is some relationship between `horsepower` and `mpg`.

**Question 2.** In 1-2 sentences describe the association between `horsepower` and `mpg` that appears in the scatter plot.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

If we only use `horsepower` alone to predict `mpg`, it appears as if we do not get good results.

Run the cell below to plot the line of best fit for the scatter plot.

In [ ]:
sns.lmplot(x = 'horsepower', y = 'mpg', data = mpg, ci = None, line_kws = {'color': 'red'})
plt.xlabel("Horsepower")
plt.ylabel("mpg");

<!-- BEGIN QUESTION -->

**Question 3.** In 1-2 sentences describe how the points are clustered around the line of best fit.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

# 2. Residuals

One one way to measure the "goodness of fit", is to plot the residuals. Residuals (also known as errors) in a statistical or machine learning model are the differences between observed and predicted values of data. 

Denoted as 

$$e_i=y_i-\hat{y}_i,$$

residuals are an important diagnostic measure that can be used to assess the quality of a model.

Residual plots can be examined in terms of their magnitude, whether or not they form a pattern, or both. Where the residuals are all 0, the model prediction and the observed value from the data set are the same. The further residuals are from 0, the less accurate the model. Where the average residual is not 0, it implies that the model is over- or under-predicting. Where residuals contain patterns, it implies that the model is qualitatively wrong, as it is failing to explain some property of the data.

A good residual plot has no pattern. This means that our model represents the relationship in the data well. If you see a curve, it is a sign that transformations of the **response or additional variables** in the model could help. A good residual plot also has a similar vertical spread throughout the entire plot. If this is not the case, the accuracy of the predictions is not reliable.

Suppose we wanted to plot the residuals of a simple linear regression model using the `galton` data set.

In [ ]:
galton = pd.read_csv("data/galton.csv")
galton.head()

In [ ]:
sns.residplot(x = "midparentHeight", y = "childHeight", data = galton)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Midparent Height")
plt.ylabel("Residuals");

What do you notice about this plot? 

The residual plot above shows a fairly random pattern. The points are randomly dispersed around the line $y=0$. This random pattern indicates that a linear model provides a decent fit to the data.

Now let's examine the residual plots of another simple linear regression model using the `tips` data set.

In [ ]:
tips = sns.load_dataset("tips")
tips.head()

In [ ]:
sns.residplot(x = "total_bill", y = "tip", data = tips)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Total Bill")
plt.ylabel("Residuals");

What do you notice about this plot?

The residuals "fan out" from left to right rather than exhibiting a consistent spread around the $y=0$ line. This violates the assumption of constant error variance. A key assumption of linear regression is that the residuals have constant variance at every level of the predictor variable.

Below, we make a residual plot of the residuals for a simple linear regression model using `horsepower` to predict `mpg`.

In [ ]:
sns.residplot(x = "horsepower", y = "mpg", data = mpg)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Horsepower")
plt.ylabel("Residuals");

Alternatively, we could have created an instance of a `LinearRegression` object, fit a model on the object, and made a scatter plot of the `horsepower` vs. the residuals.

In [ ]:
mpg_model = LinearRegression()
mpg_model.fit(mpg[['horsepower']], mpg['mpg'])
mpg_model_fitted = mpg_model.predict(mpg[['horsepower']])
plt.scatter(mpg[['horsepower']], mpg['mpg'] - mpg_model_fitted)
plt.axhline(y = 0, color = 'r')
plt.xlabel('Horsepower')
plt.ylabel('Residuals');

What do you notice about this plot?

This is not an example of a "good" residual plot. There is an underlying parabolic pattern in the residuals.

# 3. Re-expression

We should consider either using a transformation to the **response** to "straighten" the data, or think about adding a quadratic feature to the data set. For example, the code below transforms (i.e. re-expresses) the response, `mpg`, by taking the square root.

In [ ]:
mpg['sqrt_mpg'] = np.sqrt(mpg['mpg'])
mpg.head()

Now let's make a scatter plot using `horsepower` and `sqrt_mpg`.

In [ ]:
sns.lmplot(x = 'horsepower', y = 'sqrt_mpg', data = mpg, ci = None, line_kws = {'color': 'red'})
plt.ylabel('$\sqrt{mpg}$');

This looks a like a better fit. Now let's look at the residual plot.

In [ ]:
sns.residplot(x = "horsepower", y = "sqrt_mpg", data = mpg)
plt.axhline(y = 0, color = 'r')
plt.xlabel("Horsepower")
plt.ylabel("Residuals");

Now we can make a simple linear regression model using our re-expressed response `sqrt_mpg`.

In [ ]:
sqrt_mpg_model = LinearRegression()
sqrt_mpg_model.fit(mpg[['horsepower']], mpg['sqrt_mpg'])

**Question 4.** What are the coefficients for the model `sqrt_mpg_model`? In other words, given the equation for the new model is 

$$\sqrt{\text{mpg}}=\hat{a}x+\hat{b}, \hspace{1cm}\text{(1)}$$

what are the values of $\hat{a}$ and $\hat{b}$?


In [ ]:
a_hat = ...
b_hat = ...
a_hat, b_hat

In [ ]:
grader.check("q4")

<!-- BEGIN QUESTION -->

In equation **(1)** from **Question 4.** we re-expressed `mpg` as `sqrt_mpg` to straighten the data points in our scatter plot. We did this because there was a parabolic shape in our residual plot. We could leave the result in the units $\left(\sqrt{\text{mpg}}\right)$. Sometimes the new units may be as meaningful as the original, but here we want to transform the predicted value back into mpg (miles per gallon).

**Question 5.** Using the values for `a_hat` and `b_hat` (rounded to 4 decimal places), in Latex ($\LaTeX$), write out the function that the model is using to predict `mpg` from `horsepower`.

**Hint:** Click [here](http://tug.ctan.org/info/undergradmath/undergradmath.pdf) to view a brief two page guide on typesetting math equations in Latex. If you still need help, as a neighbor or your instructor.


_Type your answer here, replacing this text._

<!-- END QUESTION -->

**Question 6.** Use your equation from **Question 5.** to predict the value of `mpg` using `horsepower` for all observations in the data set. Store your result in `mpg_hat`.


In [ ]:
mpg_hat = ...

In [ ]:
grader.check("q6")

<!-- BEGIN QUESTION -->

**Question 7.** Make a residual plot using the predicted values in `mpg_hat`.


In [ ]:
plt.scatter(...)
plt.axhline(y = 0, color = 'r')
plt.xlabel('Horsepower')
plt.ylabel('Residuals');

<!-- END QUESTION -->

The dip in this residual plot is not as obvious as in our first model. 

# 3. Mean Squared Error 

The purpose of the regression line is to predict or estimate values of $y$, given values of $x$. Estimates typically aren't perfect. Each one is off the true value by an error. A reasonable criterion for a line to be the "best" is for it to have the smallest possible overall error among all possible straight lines through the data points. The mean squared error (MSE) of estimation is a measure the size squared errors. Thus minimizing mean squared error gives us the regression line. This is why the regression line is sometimes called the "least squares line".

The mean squared error is the average of the sum of the squared residuals. MSE is used to check how close model predictions are to actual values. The lower the MSE, the closer the fit is to the actual data. MSE is used as a model evaluation measure for regression models. When judging between models, generally speaking, the lowest MSE value indicates the best fit.

**Hint:** MSE is denoted 

$$\text{MSE}=\frac{1}{n}\sum\limits_{n=1}^n (y_i-\hat{y}_i)^2$$

# 4. Root Mean Square Error

Taking the square root of the mean square error yields the root mean square error (RMSE). For purposes of communicating your results to others, it is usually best to report the root mean squared error (RMSE) rather than mean squared error (MSE), because the RMSE is measured in the same units as the data, and, is representative of the size of a "typical" error. One drawback of the root mean squared error is that its sensitive to the large error. That is, the squaring process gives disproportionate weight to very large errors. 

**Hint:** RMSE is denoted 

$$\text{RMSE}=\sqrt{\frac{1}{n}\sum\limits_{n=1}^n (y_i-\hat{y}_i)^2}$$

**Question 8.** Calculate the root mean squared error for each of the fitted models, the linear model and the quadratic model.

In [ ]:
rmse_linear = ...
rmse_quad = ...
rmse_linear, rmse_quad

In [ ]:
grader.check("q8")

**Question 9.** Assign the value of the of the model with the lowest RMSE to `lowest_rmse`.

1. Linear Model
2. Quadratic Model


In [ ]:
lowest_rmse = ...

In [ ]:
grader.check("q9")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

When done exporting, download the .zip file by `SHIFT`-clicking on the file name and selecting **Save Link As**. Or, find the .zip file in the left side of the screen and right-click and select **Download**. You'll submit this .zip file for the assignment in Canvas to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)